In [13]:
import torch

from Modules import LoadingModule
from Modules import Features_encoder
from Modules import quantizationModule
from Modules import wav2vec_transformer
from Modules import ContrastiveLoss

In [9]:
#data loader module init
StandardScalerTransform = LoadingModule.StandardScalerTransform
LargeDataModule = LoadingModule.LargeDataModule("./data/Librispeech", batch_size=16, num_workers=1, transform=StandardScalerTransform)


In [14]:
#Temp import dataloader ### rendre compatible PLightning quand on aura le GPU
# en attendant import manuel
from torch.utils.data import DataLoader



dataset = TempLibriSpeech.LibriSpeech(split="test-clean", target_length=480000, device='cpu')
data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [15]:
for i, (audio, text) in enumerate(data_loader):
    print(f"Exemple {i+1}")
    print(f"Audio shape: {audio.shape}")
    print(f"Texte: {text}")
    print("-" * 50)
    if i == 1: 
        break

Exemple 1
Audio shape: torch.Size([16, 480000])
Texte: ('FORTUNATELY SAID MISTER VANDERPOOL NORTHERNERS AND SOUTHERNERS ARE ARRIVING AT A BETTER MUTUAL UNDERSTANDING ON MOST OF THESE MATTERS', 'SIR MARCH SIXTEENTH', 'SIMPLY BY STOPPING HER CARRIAGE TWO OR THREE TIMES BEFORE THE SHOP TO HAVE HER SNUFF BOX FILLED AND BY SAYING ALOUD TO THE YOUNG GIRL WHO HANDED BACK THE BOX THAT HER SNUFF WAS THE VERY BEST IN PARIS', 'I THOUGHT IT THEREFORE MY DUTY WHEN I LEFT SCHOOL TO BECOME A GOVERNESS', 'SILAS SLUNK AWAY WITHOUT A WORD OF PROTEST AMBROSE STOOD HIS GROUND EVIDENTLY BENT ON MAKING HIS PEACE WITH NAOMI BEFORE HE LEFT HER SEEING THAT I WAS IN THE WAY I WALKED ASIDE TOWARD A GLASS DOOR AT THE LOWER END OF THE ROOM', 'IN TRUTH SHE SEEMED ABSOLUTELY HIDDEN BEHIND IT', 'SAID SHE POINTING TO THE PLAYTHINGS SEE', 'SUCH CONSUMPTION AS FALLS TO THE WOMEN IS MERELY INCIDENTAL TO THEIR WORK IT IS A MEANS TO THEIR CONTINUED LABOUR AND NOT A CONSUMPTION DIRECTED TO THEIR OWN COMFORT AND FULNESS OF L

In [16]:
### Model dev ###

In [ ]:
class Model_W2V():
    def __init__(self):
        super(Model_W2V, self).__init__()
        self.FeaturesEncoder = Features_encoder.FeatureEncoder()

        self.masking = wav2vec_transformer.MaskingWithLearnableEmbedding()
        self.TranformerBlock = wav2vec_transformer.TransformerBlockW(embed_size, num_heads, dropout, forward_expansion, kernel_size, groups):

        self.quantization = quantizationModule.QuantizationModule()

        self.LossItem = ContrastiveLoss.LossW2V()

    def forward(self, x):
        x = FeaturesEncoder(x)
        
        return x
    